In [ ]:
import requests
import pandas as pd
import sys, os
import ta
import json
from ta.utils import dropna

from binance.spot import Spot

In [60]:
flag = "1"  # live trading: 0, demo trading: 1

def load_credentials(filename, API):
    with open(filename, 'r') as file:
        credentials = json.load(file)
    return credentials.get(API, None)

credentials = load_credentials('credentials.json', "API_BINANCE")

client = Spot(api_key=credentials['API_KEY'], api_secret=credentials['SECRET_KEY'])
#print(client.account())

def get_acc_balance():
    balance_dict = {}
    for item in client.account()['balances']:
        coin = item['asset']
        free = item['free']
        #locked = item['locked'] #Not necessary right now
        if float(free) != 0:
            balance_dict[coin] = free    
    return balance_dict

def get_available_trading_pairs():
    url = "https://api.binance.com/api/v3/exchangeInfo"
    response = requests.get(url)

    if response.status_code == 200:
        exchange_info = response.json()
        crypto_pairs = {}
        for item in exchange_info['symbols']:
            if "SPOT" in item['permissionSets'][0]:
                crypto_pairs[item['symbol']] = {"baseAsset": item['baseAsset'], "quoteAsset": item['quoteAsset']}
        return crypto_pairs
    else:
        print("Request failed with status code:", response.status_code)
        return None

def get_binance_kline_data(symbol, interval, limit=1000):
    url="https://api.binance.com/api/v3/klines"
    params={
        "symbol":symbol,
        "interval":interval,
        "limit":limit
    }
    try:
        response = requests.get(url,params=params)
        response.raise_for_status()
        return response
    except Exception as e:
        import traceback
        traceback.print_exc()
        print(f"Failed to fetch data for {symbol}: {e}")
        return None

#DATA FUNCTIONS:
def process_data(data):
    df =  pd.DataFrame(data, columns=["timestamp", "open", "high", "low", "close", "volume"])
    df["timestamp"]= pd.to_datetime(df["timestamp"], unit="ms")

    #Convert columns to numeric data types
    for col in ["open", "high", "low", "close", "volume"]:
        df[col]=pd.to_numeric(df[col], errors='coerce')

    #Clean NaN values
    df = dropna(df)

    #Add all ta features
    #df = add_all_ta_features(df, open="open", high="high", low="low", close="close", volume="volume")
    df = add_indicators(df)

    #Keep only the columns we're interested in
    # old version df = df[["timestamp", "open", "high", "low", "close", "volume", "momentum_rsi", "volatility_bbh", "volatility_bbl", "trend_macd", "trend_macd_signal", "trend_macd_diff"]]
    df = df[["timestamp", "open", "high", "low", "close", "volume", "rsi", "bb_high", "bb_low", "macd", "macd_signal", "macd_diff"]]      
    return df

def add_indicators(df):
    #RSI
    df["rsi"] = ta.momentum.RSIIndicator(close=df["close"], window=14, fillna=True).rsi()

    #Bollinger Bands
    bb = ta.volatility.BollingerBands(close=df["close"], window=20)
    df["bb_high"] = bb.bollinger_hband()
    df["bb_low"] = bb.bollinger_lband()

    #MACD
    macd=ta.trend.MACD(close=df["close"], window_slow=26, window_fast=12, window_sign=9)
    df["macd"]=macd.macd()
    df["macd_signal"]=macd.macd_signal()
    df["macd_diff"]=macd.macd_diff()

    #print(df)
    return df

#SIGNAL FUNCTIONS:
def generate_signals(df):
    df=df.reset_index(drop=True)

    #Initialize the variables
    df['buy_signal']=False
    df['sell_signal']=False

    #Buy when MACD crosses above the MACD signal and RSI is below 30 (oversold), and price touches lower Bollinger Band
    #Sell when MACD crosses below the MACD signal and RSI is above 70 (overbought), and price touches upper Bollinger Band
    for i in range(1, len(df)):
        if(df['macd'][i]> df['macd_signal'][i]) and (df['macd'][i-1] <= df['macd_signal'][i-1]) and (df['rsi'][i]<30) and (df['close'][i]<=df['bb_low'][i]):
            df.at[i, 'buy_signal'] = True
        elif (df['macd'][i] < df['macd_signal'][i]) and (df['macd'][i-1] >= df['macd_signal'][i-1]) and (df['rsi'][i]>70) and (df['close'][i] >= df['bb_high'][i]):
            df.at[i, 'sell_signal'] = True
    return df

def get_signals_for_pair(symbol, interval="1m", limit=1000):
    response = get_binance_kline_data(symbol, interval, limit)
    if response.status_code == 200:
        data = response.json()
    else:
        print(f"Failed to fetch data for {symbol}")
        return None
    
    if isinstance(data, list) and all(isinstance(item, list) for item in data):
        raw_data = [item[:6] for item in data]
    else:
        print(f"Unexpected data format for {symbol}")
        return None
    df = process_data(raw_data)
    df_with_indicators = add_indicators(df)
    df_with_signals = generate_signals(df_with_indicators)
    return df_with_signals[["timestamp","close","buy_signal","sell_signal"]]

def get_last_signal(symbol):
    signals = get_signals_for_pair(symbol)
    if signals is None or len(signals) == 0:
        return "hold"

    buy_signals = signals[signals["buy_signal"]]
    sell_signals = signals[signals["sell_signal"]]

    if not buy_signals.empty and not sell_signals.empty:
        last_buy_signal_index = buy_signals.index[-1]
        last_sell_signal_index = sell_signals.index[-1]
        if last_buy_signal_index > last_sell_signal_index:
            return "buy"
        else:
            return "sell"
    elif not buy_signals.empty:
        return "buy"
    elif not sell_signals.empty:
        return "sell"

    else:
        return "hold"


In [62]:
def execute_binance_strategy():
    crypto_pairs=get_available_trading_pairs()
    print("Executing my binance strategy:")
    #acc_balance = get_acc_balance()
    acc_balance = [{'asset': 'BTC', 'free': "0.1000"}, {'asset': 'ETH', 'free': "0.1000"}]
    acc_balance_currencies = []
    for balance in acc_balance:
        acc_balance_currencies.append(balance['asset'])
    print(acc_balance_currencies)
    for pair in crypto_pairs.items():
        symbol = pair[0]
        base_currency = pair[1]['baseAsset']
        quote_currency = pair[1]['quoteAsset']
        if base_currency in acc_balance_currencies or quote_currency in acc_balance_currencies:
            try:
                #print(f"Fetching signals for {symbol}")
                last_signal = get_last_signal(symbol)
                match last_signal:
                    case "buy":
                        print(f"{pair} - Buy Signal")
                        for coin, balance_str in acc_balance.items():
                            balance = float(balance_str)
                            if quote_currency in coin:
                                print(f"This is a buy signal for {symbol}")
                                #f.buy(pair, (balance*0.1))
                    case "sell":
                        print(f"{pair} - Sell Signal")
                        for coin, balance_str in acc_balance.items():
                            balance = float(balance_str)
                            if coin in pair:
                                print(f"This is a sell signal for {symbol}")
                                #f.sell(pair, balance) #We sell the 100% of the balance every time


            except Exception as e:
                import traceback
                traceback.print_exc()
                print(f"Error processing {pair}: {e}")

execute_binance_strategy()

Executing my binance strategy:
['BTC', 'ETH']


In [ ]:
crypto_pairs=get_available_trading_pairs()
print("Executing my binance strategy:")
print(len(crypto_pairs))
"""for pair in crypto_pairs.items():
    symbol = pair[0]
    last_signal = get_signals_for_pair(symbol)
    #print(f"{symbol} - {last_signal}")"""